<a href="https://colab.research.google.com/github/yamanoyu/atcorder/blob/master/_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Combination

In [ ]:
def cmb(n, r):
    # Combinationを算出 その1
    # 5C2 = 10など
    if n - r < r: r = n - r
    if r == 0: return 1
    if r == 1: return n

    numerator = [n - r + k + 1 for k in range(r)]
    denominator = [k + 1 for k in range(r)]

    for p in range(2, r+1):
        pivot = denominator[p - 1]
        if pivot > 1:
            offset = (n - r) % p
            for k in range(p-1, r, p):
                numerator[k - offset] /= pivot
                denominator[k] /= pivot
    result = 1
    for k in range(r):
        if numerator[k] > 1:
            result *= int(numerator[k])

    return result

In [ ]:
from operator import mul
from functools import reduce

def combination_cnt(n, r):
  # Combinationを算出 その2 (遅い)
  # 5C2 = 10など

  r = min(n - r, r)
  if r == 0: return 1
  return reduce(mul, range(n - r + 1, n + 1)) // reduce(mul, range(1, r + 1))

## 素因数分解

In [ ]:
def prime_factorization(n):
    # 素因数分解 その1
    # prime_factorization(24) の場合 [[2, 3], [3, 1]] (s^3 * 3^1)を返す
    
    arr = []
    tmp = n
    for i in range(2, int((-(-(n ** 0.5)) // 1)) + 1):
        if tmp % i == 0:
            cnt = 0
            while tmp % i == 0:
                cnt += 1
                tmp //= i
            arr.append([i, cnt])

    if tmp != 1:
        arr.append([tmp, 1])

    if arr == []:
        arr.append([n, 1])

    return arr


In [ ]:
def factorization(n):
  # 素因数分解 その2
  # prime_factorization(24) の場合 [[2, 3], [3, 1]] (s^3 * 3^1)を返す
  if n == 1: return []
  factorization_list = []
  i = 2
  _n = n
  while i ** 2 <= n:
    cnt = 0
    while _n % i == 0:
      cnt += 1
      _n //= i
    factorization_list.append([i, cnt])
    i += 1
  
  if _n != 1: factorization_list.append([_n, 1])
  if factorization_list == []: factorization_list.append([n, 1])

  return factorization_list

## フィボナッチ数列

In [ ]:
# これだとすごく遅い
# def fibonacci_sequence(n):
#   if n <= 2:
#     return 1
#   else:
#     return fibonacci_sequence(n - 1) + fibonacci_sequence(n - 2)

# 動的計画法を利用する
def fibonacci_sequence(n):
  fibonacci_sequence_list = [None] * n

  if n == 0:
    return 0
  elif n == 1 or n == 2:
    return 1
  else: 
    fibonacci_sequence_list[0] = 1
    fibonacci_sequence_list[1] = 1
    for i in range(2, n):
      fibonacci_sequence_list[i] = fibonacci_sequence_list[i - 1] + fibonacci_sequence_list[i - 2]

  return fibonacci_sequence_list[n - 1]

fibonacci_sequence(100)

354224848179261915075

## 素数

In [ ]:
# エラトステネスのふるい

def sieve_of_eratosthenes(n):
    # n以下の素数を出力
    prime_num_list = [2]
    number_list = [num for num in range(3, n, 2)]

    while True:
      p = number_list[0]
      if p ** 2 > n:
        return prime_num_list + number_list
        break
      prime_num_list.append(p)
      number_list = [num for num in number_list if num % p != 0]

sieve_of_eratosthenes(100)

[2,
 3,
 5,
 7,
 11,
 13,
 17,
 19,
 23,
 29,
 31,
 37,
 41,
 43,
 47,
 53,
 59,
 61,
 67,
 71,
 73,
 79,
 83,
 89,
 97]

## UnionFind

In [ ]:
class UnionFind:
    # if x is root: self.parents[x] = - (the number of the group nodes)
    # else: self.parents[x] = the parent of x
    def __init__(self, n):
        self.n = n
        self.parents = [-1] * n
 
    # return the parent of x
    def find(self, x):
        history = []
        while self.parents[x] >= 0:
            history.append(x)
            x = self.parents[x]
        for node in history:
            self.parents[node] = x
        return x
 
    # merge the group of x and the group of y
    def union(self, x, y):
        x = self.find(x)
        y = self.find(y)
        if x == y:
            return
        if self.parents[x] > self.parents[y]:
            x, y = y, x
        self.parents[x] += self.parents[y]
        self.parents[y] = x
 
    # return the size of the group of x
    def size(self, x):
        return -self.parents[self.find(x)]
 
    # return whether x and y in a same group
    def same(self, x, y):
        return self.find(x) == self.find(y)
 
    # return [all roots]
    # O(n)
    def roots(self):
        return [i for i, x in enumerate(self.parents) if x < 0]
 
    # return the number of groups
    # O(n)
    def group_count(self):
        return len(self.roots())

## 転倒数

In [ ]:
class Bit:
  # 転倒数
  def __init__(self, n):
    self.size = n
    self.tree = [0] * (n + 1)

  def __iter__(self):
      psum = 0
      for i in range(self.size):
          csum = self.sum(i + 1)
          yield csum - psum
          psum = csum
      raise StopIteration()

  def __str__(self):  # O(nlogn)
      return str(list(self))

  def sum(self, i):
      # [0, i) の要素の総和を返す
      if not (0 <= i <= self.size): raise ValueError("error!")
      s = 0
      while i>0:
          s += self.tree[i]
          i -= i & - i
      return s

  def add(self, i, x):
      if not (0 <= i < self.size): raise ValueError("error!")
      i += 1
      while i <= self.size:
          self.tree[i] += x
          i += i & - i

  def __getitem__(self, key):
      if not (0 <= key < self.size): raise IndexError("error!")
      return self.sum(key+1) - self.sum(key)

  def __setitem__(self, key, value):
      # 足し算と引き算にはaddを使うべき
      if not (0 <= key < self.size): raise IndexError("error!")
      self.add(key, value - self[key])